In [3]:
pip install geocoder

     |████████████████████████████████| 98 kB 8.3 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
 pip install folium

     |████████████████████████████████| 94 kB 5.5 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


2. SCRAP WIKIPEDIA INTO DF 

In [54]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Vancouver").text

In [55]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [56]:
# create a list to store neighborhood data
neighborhoodList = []

In [64]:
# create a new DataFrame from the list
va_df = pd.DataFrame({"Neighborhood": neighborhoodList})

va_df.head()
    

,Neighborhood
0,Arbutus Ridge
1,Champlain Heights
2,"Chinatown, Vancouver"
3,Coal Harbour
4,Concord Pacific Place


In [66]:
# print the number of rows of the dataframe
va_df.shape

(45, 1)

3. OBTAINGING THE LONGITUDE AND LATITUDE COORDINATES 

In [67]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Vancouver, Canada'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [69]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in va_df["Neighborhood"].tolist() ]

In [70]:
coords

[[49.24936000000008, -123.16533999999996],
 [49.213640000000055, -123.03751999999997],
 [49.279930000000036, -123.09971999999999],
 [49.29120000000006, -123.12964999999997],
 [49.27627241121121, -123.11006317810512],
 [49.285681029493205, -123.13939885234531],
 [49.283930000000055, -123.09044999999998],
 [49.28108000000003, -123.11589999999995],
 [49.249960000000044, -123.18050999999997],
 [49.26621033791696, -123.25086146214669],
 [49.260380000000055, -123.11335999999994],
 [49.26613000000003, -123.13322999999997],
 [49.27173000000005, -123.13611999999995],
 [49.260380000000055, -123.11335999999994],
 [49.28364000000005, -123.10645999999997],
 [49.28296000000006, -123.06933999999995],
 [43.64791000000008, -79.39080999999999],
 [49.27036000000004, -123.13423999999998],
 [49.260380000000055, -123.11335999999994],
 [49.28021000000007, -123.02868999999998],
 [49.260380000000055, -123.11335999999994],
 [49.260380000000055, -123.11335999999994],
 [49.249730000000056, -123.06985999999995],
 

In [71]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [72]:
# merge the coordinates into the original dataframe
va_df['Latitude'] = df_coords['Latitude']
va_df['Longitude'] = df_coords['Longitude']

In [73]:
# check the neighborhoods and the coordinates
print(va_df.shape)
va_df

(45, 3)


,Neighborhood,Latitude,Longitude
0,Arbutus Ridge,49.249360,-123.165340
1,Champlain Heights,49.213640,-123.037520
2,"Chinatown, Vancouver",49.279930,-123.099720
3,Coal Harbour,49.291200,-123.129650
4,Concord Pacific Place,49.276272,-123.110063
5,Davie Village,49.285681,-123.139399
6,Downtown Eastside,49.283930,-123.090450
7,Downtown Vancouver,49.281080,-123.115900
8,Dunbar–Southlands,49.249960,-123.180510
9,East Vancouver,49.266210,-123.250861


In [74]:
# save the DataFrame as CSV file
va_df.to_csv("kl_df.csv", index=False)

4. CREATING A VISUAL MAP OF VANCOUVER WITH ALL THE NEIGHBORHOODS 

In [81]:
address = 'Vancouver, Canada'
geolocator = Nominatim(user_agent="http")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver, Canada {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver, Canada 49.2608724, -123.1139529.


In [83]:
# create map of Vancouver using latitude and longitude values
map_va = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(va_df['Latitude'], va_df['Longitude'], va_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_va)  
    
map_va

In [84]:
# save the map as HTML file
map_va.save('map_va.html')

5. UTILIZE FOURSQUARE API 

In [85]:
CLIENT_ID = 'PHWZG1WVQKVVPXD03QGCQ0V4BMPTORHWEUA1GTWZEKNFNC14'
CLIENT_SECRET = 'P2OTAJQLRSOXMHS5LDRMTL2MQBTDV3RNHWPDRZ0N25YZT1J0'
VERSION = '20180605'
radius = 2000
LIMIT = 100
venues = []
for lat, long, neighborhood in zip(va_df['Latitude'], va_df['Longitude'], va_df['Neighborhood']):
  url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,long,radius,LIMIT)
  results = requests.get(url).json()["response"]['groups'][0]['items']
  for venue in results:
      venues.append((neighborhood,lat,long,venue['venue']['name'],
      venue['venue']['location']['lat'],venue['venue']['location']    ['lng'],venue['venue']['categories'][0]['name']))

In [86]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(4333, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Arbutus Ridge,49.24936,-123.16534,The Patty Shop,49.250680,-123.167916,Caribbean Restaurant
1,Arbutus Ridge,49.24936,-123.16534,La Buca,49.250549,-123.167933,Italian Restaurant
2,Arbutus Ridge,49.24936,-123.16534,Butter Baked Goods,49.242209,-123.170381,Bakery
3,Arbutus Ridge,49.24936,-123.16534,Sweet Obsession Cakes & Pastries,49.257756,-123.165314,Dessert Shop
4,Arbutus Ridge,49.24936,-123.16534,The Arbutus Club,49.248507,-123.152152,Event Space


In [87]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Arbutus Ridge,91,91,91,91,91,91
Champlain Heights,47,47,47,47,47,47
"Chinatown, Vancouver",100,100,100,100,100,100
Coal Harbour,100,100,100,100,100,100
Concord Pacific Place,100,100,100,100,100,100
Davie Village,100,100,100,100,100,100
Downtown Eastside,100,100,100,100,100,100
Downtown Vancouver,100,100,100,100,100,100
Dunbar–Southlands,100,100,100,100,100,100


In [88]:

print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 241 uniques categories.


In [89]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Caribbean Restaurant', 'Italian Restaurant', 'Bakery',
       'Dessert Shop', 'Event Space', 'Park', 'Pizza Place', 'Café',
       'Grocery Store', 'Japanese Restaurant', 'Coffee Shop',
       'Yoga Studio', 'Portuguese Restaurant', 'Sushi Restaurant',
       'Middle Eastern Restaurant', 'Greek Restaurant', 'Spa',
       'Gym / Fitness Center', 'Donut Shop', 'Tennis Court', 'Tea Room',
       'Bookstore', 'Vegetarian / Vegan Restaurant', 'Pet Store',
       'Burger Joint', 'Chinese Restaurant', 'Indie Movie Theater',
       'Indian Restaurant', 'French Restaurant', 'Pharmacy',
       'Vietnamese Restaurant', 'Mediterranean Restaurant', 'Bank',
       'Sandwich Place', 'Discount Store', 'Gym', 'Ice Cream Shop',
       'Boutique', 'Business Service', 'Cosmetics Shop', 'Noodle House',
       'Gym Pool', 'Liquor Store', 'Restaurant', 'Pub',
       'Convenience Store', 'Shopping Mall', 'Basketball Court',
       'Asian Restaurant', 'Seafood Restaurant'], dtype=object)

In [90]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

6. FURTHER ANALYZING EACH NEIGHBOURHOOD 

In [91]:
# one hot encoding
va_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
va_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [va_onehot.columns[-1]] + list(va_onehot.columns[:-1])
va_onehot = va_onehot[fixed_columns]

print(va_onehot.shape)
va_onehot.head()

(4333, 242)


,Neighborhoods,Accessories Store,American Restaurant,Amphitheater,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hot Dog Joint,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trade School,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Water Park,Waterfront,Wine Shop,Women's Store,Yoga Studio
0,Arbutus Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Arbutus Ridge,0,0,0,0,0,0,0,0,0,0,0

In [92]:
va_grouped = va_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(va_grouped.shape)
va_grouped

(45, 242)


,Neighborhoods,Accessories Store,American Restaurant,Amphitheater,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hot Dog Joint,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trade School,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Water Park,Waterfront,Wine Shop,Women's Store,Yoga Studio
0,Arbutus Ridge,0.00,0.000000,0.00,0.00,0.00,0.00,0.010989,0.000000,0.00,0.00,0.00,0.010989,0.032967,0.000000,0.00,0.000000,0.010989,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.010989,0.000000,0.010989,0.00,0.00,0.00,0.00000,0.000000,0.00,0.010989,0.00,0.00,0.00000,0.000000,0.010989,0.021978,0.00,0.00,0.00,0.000000,0.010989,0.00,0.00,0.021978,0.000000,0.00,0.00,0.131868,0.00,0.00,0.000000,0.010989,0.010989,0.00,0.00,0.00,0.000000,0.032967,0.000000,0.00000,0.010989,0.000000,0.010989,0.000000,0.00,0.00,0.010989,0.00,0.00

In [93]:
len(va_grouped[va_grouped["Shopping Mall"] > 0])

9

In [94]:
va_mall = va_grouped[["Neighborhoods","Shopping Mall"]]

In [95]:

va_mall.head()

,Neighborhoods,Shopping Mall
0,Arbutus Ridge,0.010989
1,Champlain Heights,0.021277
2,"Chinatown, Vancouver",0.000000
3,Coal Harbour,0.000000
4,Concord Pacific Place,0.000000


7. CLUSTERING NEIGHBOURHOODS 

In [96]:
# set number of clusters
kclusters = 3

va_clustering = va_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(va_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 0, 0, 0, 0, 0, 2, 0, 2], dtype=int32)

In [97]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
va_merged = va_mall.copy()

# add clustering labels
va_merged["Cluster Labels"] = kmeans.labels_

In [98]:
va_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
va_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Arbutus Ridge,0.010989,2
1,Champlain Heights,0.021277,1
2,"Chinatown, Vancouver",0.000000,0
3,Coal Harbour,0.000000,0
4,Concord Pacific Place,0.000000,0


In [99]:
# merge vancouver_grouped with vancouver_data to add latitude/longitude for each neighborhood
va_merged = va_merged.join(va_df.set_index("Neighborhood"), on="Neighborhood")

print(va_merged.shape)
va_merged.head() # check the last columns!

(45, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Arbutus Ridge,0.010989,2,49.249360,-123.165340
1,Champlain Heights,0.021277,1,49.213640,-123.037520
2,"Chinatown, Vancouver",0.000000,0,49.279930,-123.099720
3,Coal Harbour,0.000000,0,49.291200,-123.129650
4,Concord Pacific Place,0.000000,0,49.276272,-123.110063


In [100]:
# sort the results by Cluster Labels
print(va_merged.shape)
va_merged.sort_values(["Cluster Labels"], inplace=True)
va_merged

(45, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
22,Kensington–Cedar Cottage,0.000000,0,49.249730,-123.069860
23,Kerrisdale,0.000000,0,49.220970,-123.150730
25,Kitsilano,0.000000,0,49.268330,-123.165420
26,"Little Italy, Vancouver",0.000000,0,49.242165,-123.064000
27,Marpole,0.000000,0,49.200670,-123.130530
28,"Mount Pleasant, Vancouver",0.000000,0,49.261780,-123.096080
29,Musqueam Indian Reserve No. 2,0.000000,0,49.260380,-123.113360
31,"Punjabi Market, Vancouver",0.000000,0,49.285550,-123.126330
32,Renfrew–Collingwood,0.000000,0,49.249700,-123.029750
33,Riley Park–Little Mountain,0.000000,0,49.242165,-123.064000


In [101]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(va_merged['Latitude'], va_merged['Longitude'], va_merged['Neighborhood'], va_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [102]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

8. EXAMINATION OF CLUSTERS 

In [104]:
va_merged.loc[va_merged['Cluster Labels'] == 0]
#cluster 0

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
22,Kensington–Cedar Cottage,0.0,0,49.249730,-123.069860
23,Kerrisdale,0.0,0,49.220970,-123.150730
25,Kitsilano,0.0,0,49.268330,-123.165420
26,"Little Italy, Vancouver",0.0,0,49.242165,-123.064000
27,Marpole,0.0,0,49.200670,-123.130530
28,"Mount Pleasant, Vancouver",0.0,0,49.261780,-123.096080
29,Musqueam Indian Reserve No. 2,0.0,0,49.260380,-123.113360
31,"Punjabi Market, Vancouver",0.0,0,49.285550,-123.126330
32,Renfrew–Collingwood,0.0,0,49.249700,-123.029750
33,Riley Park–Little Mountain,0.0,0,49.242165,-123.064000


In [106]:
va_merged.loc[va_merged['Cluster Labels'] == 1]
#cluster 1

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
1,Champlain Heights,0.021277,1,49.21364,-123.03752


In [107]:
va_merged.loc[va_merged['Cluster Labels'] == 2]
#cluster 2

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
9,East Vancouver,0.013333,2,49.26621,-123.250861
14,Gastown,0.010000,2,49.28364,-123.106460
7,Downtown Vancouver,0.010000,2,49.28108,-123.115900
16,Granville Entertainment District,0.010000,2,43.64791,-79.390810
24,"Killarney, Vancouver",0.012048,2,49.22670,-123.036570
41,"Victory Square, Vancouver",0.010000,2,49.28200,-123.110690
30,"Oakridge, Vancouver",0.012195,2,49.23004,-123.121170
0,Arbutus Ridge,0.010989,2,49.24936,-123.165340


OBSERVATION OF RESULTS 

We can see that cluster 0 has no malls in the area, where as cluster 1 and cluster 2 has the highest and moderate number of malls respectively. This means that there is a massive opportunity to build a mall in the area of cluster 0 as there will be no competition and will most likey draw the population from around there to visit as they will not have to travel to clusters 1 or 2. Since travel time to get to malls for these customers in cluster 0 will be shortned, this could also reduce the amount of online shopping they decide to do and further increase traffic to the newly constructed mall if built. Therefore, it would be a good idea to build a mall in cluster 0. 